### Straddle

## variable declaration

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import os

pd.set_option('display.max_columns',None)

# Set the display width option
pd.set_option('display.width', 1000)


date = '2022-03-10'      
datestr = date.split('-')[0]
start_time = '09:15'
entry_time = '09:20'
exit_time = '15:15'
end_time = '15:20'
entry_side = 'sell'

underlying = 'BANKNIFTY'
underlying_strike_diff = 100 if underlying == 'BANKNIFTY' else 50
OTM_points = 100
exp_period = 'currWeek_exp_dt'

Sl_pct = 0.2  #0.2 -> 20%
tgt_pct = 0.8  #0.8 --> 80%

#straddle variables
straddle_SL = 1
straddle_Tgh = 1

#conversion
date_dt = pd.to_datetime(date).date()
entry_time_dt = pd.to_datetime(entry_time).time()
exit_time_dt = pd.to_datetime(exit_time).time()
end_time_dt = pd.to_datetime(end_time).time()

## reading bankNifty file

In [2]:
bank_nifty_df = pd.read_csv('C:\\keshav\\Rahul\\Options_data\\NIFTY & BANKNIFTY (Jan 2020 to 19 Dec 2022)\\NIFTY & BANKNIFTY Spot Indices (Jan 2020 to 19 Dec 2022)\\BANKNIFTY.csv',header=None)
bank_nifty_df.rename(columns={
                0 : 'date',
                1 : 'time',
                2 : 'open',
                3 : 'high',
                4 : 'low',
                5 : 'close'
},inplace=True)
bank_nifty_df['date'] = bank_nifty_df['date'].astype('str')
datetime = pd.to_datetime(bank_nifty_df['date'] + ' ' + bank_nifty_df['time'], format='%Y%m%d %H:%M')
bank_nifty_df.insert(1,'datetime',datetime) 
bank_nifty_df['date'] = pd.to_datetime(bank_nifty_df['date'])
bank_nifty_df.set_index('datetime',inplace=True)
bank_nifty_df= bank_nifty_df.between_time('09:15:00', '15:29:00')
bank_nifty_df

,date,time,open,high,low,close
datetime,,,,,,
2020-01-01 09:15:00,2020-01-01,09:15,32227.80,32295.80,32220.90,32292.00
2020-01-01 09:16:00,2020-01-01,09:16,32296.35,32303.30,32283.95,32288.25
2020-01-01 09:17:00,2020-01-01,09:17,32285.05,32290.85,32279.35,32282.95
2020-01-01 09:18:00,2020-01-01,09:18,32280.20,32296.25,32270.70,32292.45
2020-01-01 09:19:00,2020-01-01,09:19,32295.30,32312.05,32294.85,32298.65
...,...,...,...,...,...,...
2022-12-19 15:25:00,2022-12-19,15:25,43432.50,43433.35,43421.35,43429.75
2022-12-19 15:26:00,2022-12-19,15:26,43428.05,43435.30,43425.95,43433.90
2022-12-19 15:27:00,2022-12-19,15:27,43436.65,43451.45,43432.80,43444.60


### extracting unique ordered date list out of banknifty

In [3]:
import pandas as pd
x = sorted(list(set(bank_nifty_df.index.date)))
x = pd.to_datetime(x)

unique_date_list = [date.strftime("%Y-%m-%d") for date in x]

### unique dates from banknifty_df

## Reading exp detail file

In [4]:
weekly_exp_df = pd.read_csv("C:\keshav\Rahul\Options_data\Weekly_exp_dates.csv", usecols=['Exp_date'])

weekly_exp_df['Exp_date'] = pd.to_datetime(weekly_exp_df['Exp_date'], format='%d-%b-%y')
weekly_exp_df['day'] = weekly_exp_df['Exp_date'].dt.day
weekly_exp_df['monthly_exp'] = False
weekly_exp_df


,Exp_date,day,monthly_exp
0,2020-01-02,2,False
1,2020-01-09,9,False
2,2020-01-16,16,False
3,2020-01-23,23,False
4,2020-01-30,30,False
...,...,...,...
150,2022-11-17,17,False
151,2022-11-24,24,False
152,2022-12-01,1,False
153,2022-12-08,8,False


## making merged_df dataframe with all expiry dates --> merged_df

In [5]:
for i in range(len(weekly_exp_df)):
    try:
        if weekly_exp_df.loc[i, 'day'] > weekly_exp_df.loc[i+1, 'day']:
            weekly_exp_df.loc[i, 'monthly_exp'] = True
        monthly_exp_df = pd.DataFrame()
        monthly_exp_df = weekly_exp_df.loc[(weekly_exp_df['monthly_exp'] == True), ['Exp_date', 'day']]
        df_weekly_list = weekly_exp_df[['Exp_date']]
        df_monthly_list = monthly_exp_df[['Exp_date']]

        #making newe dataframe
        df_datelist = pd.DataFrame(bank_nifty_df['date'].unique(), columns = ['date'])
        df_datelist[['currWeek_exp_dt', 'nextWeek_exp_dt', 'farWeek_exp_dt','currMonth_exp_dt', 'nextMonth_exp_dt', 'farMonth_exp_dt' ]] = np.NaN

    except: 
        pass

df_datelist

,date,currWeek_exp_dt,nextWeek_exp_dt,farWeek_exp_dt,currMonth_exp_dt,nextMonth_exp_dt,farMonth_exp_dt
0,2020-01-01,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-02,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-03,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01-06,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-07,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
731,2022-12-13,NaN,NaN,NaN,NaN,NaN,NaN
732,2022-12-14,NaN,NaN,NaN,NaN,NaN,NaN
733,2022-12-15,NaN,NaN,NaN,NaN,NaN,NaN
734,2022-12-16,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
for i in range(len(df_datelist)):
    try:
        df2 = df_weekly_list.loc[df_weekly_list['Exp_date'] >= df_datelist['date'][i]]
        df_datelist.loc[i,'currWeek_exp_dt'] = df2['Exp_date'].values[0]
        df_datelist.loc[i,'nextWeek_exp_dt'] = df2['Exp_date'].values[1]
        df_datelist.loc[i,'farWeek_exp_dt'] = df2['Exp_date'].values[2]
        df3 = df_monthly_list.loc[df_monthly_list['Exp_date'] >= df_datelist['date'][i]]

        df_datelist.loc[i,'currMonth_exp_dt'] = df3['Exp_date'].values[0]
        df_datelist.loc[i,'nextMonth_exp_dt'] = df3['Exp_date'].values[1]
        df_datelist.loc[i,'farMonth_exp_dt'] = df3['Exp_date'].values[2]
    
    except:
        pass    

merged_df = pd.merge(bank_nifty_df,df_datelist,how='left', on='date')
merged_df['currWeek_exp_dt'] = pd.to_datetime(merged_df['currWeek_exp_dt']).dt.date
merged_df['nextWeek_exp_dt'] = pd.to_datetime(merged_df['nextWeek_exp_dt']).dt.date
merged_df['farWeek_exp_dt'] = pd.to_datetime(merged_df['farWeek_exp_dt']).dt.date
merged_df['currMonth_exp_dt'] = pd.to_datetime(merged_df['currMonth_exp_dt']).dt.date
merged_df['nextMonth_exp_dt'] = pd.to_datetime(merged_df['nextMonth_exp_dt']).dt.date
merged_df['farMonth_exp_dt'] = pd.to_datetime(merged_df['farMonth_exp_dt']).dt.date

merged_df


,date,time,open,high,low,close,currWeek_exp_dt,nextWeek_exp_dt,farWeek_exp_dt,currMonth_exp_dt,nextMonth_exp_dt,farMonth_exp_dt
0,2020-01-01,09:15,32227.80,32295.80,32220.90,32292.00,2020-01-02,2020-01-09,2020-01-16,2020-01-30,2020-02-27,2020-03-26
1,2020-01-01,09:16,32296.35,32303.30,32283.95,32288.25,2020-01-02,2020-01-09,2020-01-16,2020-01-30,2020-02-27,2020-03-26
2,2020-01-01,09:17,32285.05,32290.85,32279.35,32282.95,2020-01-02,2020-01-09,2020-01-16,2020-01-30,2020-02-27,2020-03-26
3,2020-01-01,09:18,32280.20,32296.25,32270.70,32292.45,2020-01-02,2020-01-09,2020-01-16,2020-01-30,2020-02-27,2020-03-26
4,2020-01-01,09:19,32295.30,32312.05,32294.85,32298.65,2020-01-02,2020-01-09,2020-01-16,2020-01-30,2020-02-27,2020-03-26
...,...,...,...,...,...,...,...,...,...,...,...,...
275481,2022-12-19,15:25,43432.50,43433.35,43421.35,43429.75,NaN,NaN,NaN,NaN,NaN,NaN
275482,2022-12-19,15:26,43428.05,43435.30,43425.95,43433.90,NaN,NaN,NaN,NaN,NaN,NaN
275483,2022-12-19,15:27,43436.65,43451.45,43432.80,43444.60,NaN,NaN,NaN,NaN,NaN,NaN
275484,2022-12-19,15:28,43446.15,43448.25,43422.00,43428.60,NaN,NaN,NaN,NaN,NaN,NaN


### formatting merged_df dataframe

In [7]:
merged_df['date'] = merged_df['date'].astype(str)
merged_df['time'] = merged_df['time'].astype(str)
datetime = pd.to_datetime(merged_df['date'] + ' ' + merged_df['time'], format='mixed', dayfirst=True)
bank_nifty_df.index = pd.to_datetime(bank_nifty_df.index)
merged_df.insert(0,'datetime',datetime) 
merged_df.set_index('datetime',inplace=True)
merged_df.drop(columns=['date','time'],inplace=True)

merged_df

,open,high,low,close,currWeek_exp_dt,nextWeek_exp_dt,farWeek_exp_dt,currMonth_exp_dt,nextMonth_exp_dt,farMonth_exp_dt
datetime,,,,,,,,,,
2020-01-01 09:15:00,32227.80,32295.80,32220.90,32292.00,2020-01-02,2020-01-09,2020-01-16,2020-01-30,2020-02-27,2020-03-26
2020-01-01 09:16:00,32296.35,32303.30,32283.95,32288.25,2020-01-02,2020-01-09,2020-01-16,2020-01-30,2020-02-27,2020-03-26
2020-01-01 09:17:00,32285.05,32290.85,32279.35,32282.95,2020-01-02,2020-01-09,2020-01-16,2020-01-30,2020-02-27,2020-03-26
2020-01-01 09:18:00,32280.20,32296.25,32270.70,32292.45,2020-01-02,2020-01-09,2020-01-16,2020-01-30,2020-02-27,2020-03-26
2020-01-01 09:19:00,32295.30,32312.05,32294.85,32298.65,2020-01-02,2020-01-09,2020-01-16,2020-01-30,2020-02-27,2020-03-26
...,...,...,...,...,...,...,...,...,...,...
2022-12-19 15:25:00,43432.50,43433.35,43421.35,43429.75,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-19 15:26:00,43428.05,43435.30,43425.95,43433.90,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-19 15:27:00,43436.65,43451.45,43432.80,43444.60,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
merged_df.insert(4,'ATMStrPr',round(merged_df['close'],-2))

In [9]:
merged_df

,open,high,low,close,ATMStrPr,currWeek_exp_dt,nextWeek_exp_dt,farWeek_exp_dt,currMonth_exp_dt,nextMonth_exp_dt,farMonth_exp_dt
datetime,,,,,,,,,,,
2020-01-01 09:15:00,32227.80,32295.80,32220.90,32292.00,32300.0,2020-01-02,2020-01-09,2020-01-16,2020-01-30,2020-02-27,2020-03-26
2020-01-01 09:16:00,32296.35,32303.30,32283.95,32288.25,32300.0,2020-01-02,2020-01-09,2020-01-16,2020-01-30,2020-02-27,2020-03-26
2020-01-01 09:17:00,32285.05,32290.85,32279.35,32282.95,32300.0,2020-01-02,2020-01-09,2020-01-16,2020-01-30,2020-02-27,2020-03-26
2020-01-01 09:18:00,32280.20,32296.25,32270.70,32292.45,32300.0,2020-01-02,2020-01-09,2020-01-16,2020-01-30,2020-02-27,2020-03-26
2020-01-01 09:19:00,32295.30,32312.05,32294.85,32298.65,32300.0,2020-01-02,2020-01-09,2020-01-16,2020-01-30,2020-02-27,2020-03-26
...,...,...,...,...,...,...,...,...,...,...,...
2022-12-19 15:25:00,43432.50,43433.35,43421.35,43429.75,43400.0,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-19 15:26:00,43428.05,43435.30,43425.95,43433.90,43400.0,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-19 15:27:00,43436.65,43451.45,43432.80,43444.60,43400.0,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
def CE_strike_prices(entry_time_dt,date_dt):
    return str(round(merged_df.loc[(merged_df.index.time == entry_time_dt) & (merged_df.index.date == date_dt), 'ATMStrPr'].values[0] + OTM_points))


def PE_strike_prices(entry_time_dt,date_dt):
    return str(round(merged_df.loc[(merged_df.index.time == entry_time_dt) & (merged_df.index.date == date_dt), 'ATMStrPr'].values[0] - OTM_points))

CE_stp = CE_strike_prices(entry_time_dt,date_dt)
PE_stp = PE_strike_prices(entry_time_dt,date_dt)

In [11]:
merged_df['CE_strike_price'] = CE_stp
merged_df['PE_strike_price'] = PE_stp

In [13]:
def exp_date_loc(entry_time_dt,date_dt):
    result= merged_df.loc[(merged_df.index.time == entry_time_dt) & (merged_df.index.date == date_dt)][exp_period]
    result = result.values[0]
    date = pd.to_datetime(result)
    formatted_date = date.strftime('%y%m%d')
    return formatted_date

exp_date = exp_date_loc(entry_time_dt,date_dt)

ce_symbol = underlying+exp_date+CE_stp+'CE'
ce_symbol = f"{ce_symbol}"

pe_symbol = underlying+exp_date+PE_stp+'PE'
pe_symbol = f"{pe_symbol}"


ce_symbol
pe_symbol


'BANKNIFTY22031034900PE'